In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
cpu_price = pd.DataFrame(data=[['amazon', 4, 0.17], 
                           ['amazon', 8, 0.34],
                           ['amazon', 16, 0.68],
                           ['amazon', 36, 1.53],
                          ['google', 4, 0.2088],
                          ['google', 8, 0.4176],
                          ['google', 16, 0.8325],
                          ['microsoft', 4, 0.17],
                          ['microsoft', 8, 0.34]],
                    columns=['provider', 'vcpu(s)','price'])

In [3]:
cpu_price

,provider,vcpu(s),price
0,amazon,4,0.1700
1,amazon,8,0.3400
2,amazon,16,0.6800
3,amazon,36,1.5300
4,google,4,0.2088
5,google,8,0.4176
6,google,16,0.8325
7,microsoft,4,0.1700
8,microsoft,8,0.3400


In [4]:
gpu_price = pd.DataFrame(data=[['amazon', 0.9],
                              ['google', 0.481]],
                        columns=['provider','price'])

In [5]:
OUTPUT_DIR = "outputs/"

In [6]:
def insert(df, record):
    idx = df.shape[0] + 1
    df.loc[idx] = record

In [7]:
def summarize_exp(exp):
    exp = exp.rename(columns=exp.iloc[0]).iloc[1:, :]
    for col in exp:
        exp[col] = exp[col].astype('float')
    exp = exp.replace([np.inf, -np.inf], np.nan).dropna(how='any')
    # return peak GFLOPS (throughput)
    return "%.4f"%exp.max()[2]
def process_gpu_benchmark(filename):
    f = open(OUTPUT_DIR + filename)
    provider, device, other = filename.split('_')
    mode, other = other.split('-')
    instance = other.split('.')[0]
    iteration = 0
    start_csv = False
    csv = []
    for line in f:
        if "Experiment ID" in line:
            start_csv = True
        elif "-------------" in line:
            if start_csv:
                start_csv = False
                # finish scanning a csv file
                result = pd.DataFrame(csv)
                record = [summarize_exp(result.iloc[:, i*4+1:i*4+5]) for i in [0, 1, 3]]
                iteration += 1
                insert(gpu, [filename, provider, device, instance, iteration] + record)
                csv = []
        elif start_csv:
            csv.append(line.strip().split(','))

In [8]:
def process_cpu_benchmark(filename):
    f = open(OUTPUT_DIR + filename)
    provider, vcpu, other = filename.split('_')
    device, other = other.split('-')
    instance = other.split('.')[0]
    iteration = 0
    for line in f:
        if "total time:" in line:
            time = float(line.split(':')[-1].split('s')[0].strip()) * 1000
        elif "total number of events:" in line:
            events = int(line.split(':')[-1].strip())
            time_per_task = time / events
            iteration += 1
            insert(cpu, [filename, provider, vcpu, device, instance, iteration, events, time, time_per_task])

In [9]:
def process_ml_benchmark(filename):
    f = open(OUTPUT_DIR + filename)
    provider, vcpu, device, other = filename.split('_')
    instance = other.split('-')[-1].split('.')[0]
    iteration = 0
    for line in f:
        if "Accuracy:" in line:
            accuracy = eval(line.strip().split('Accuracy: ')[1].split()[0])
        elif "Total training completion time" in line:
            iteration += 1
            time = line.split()[-2]
            if device == "cpu":
                insert(cpu_ml, [filename, provider, vcpu, device, instance, iteration, accuracy, time])
            else:
                insert(gpu_ml, [filename, provider, vcpu, device, instance, iteration, accuracy, time])

In [10]:
cpu = pd.DataFrame(columns=['filename', 'provider', 'vcpu(s)', 'device', 'instance', 'iteration', 'events', 'runtime(ms)', 'time_per_task(ms)'])
gpu = pd.DataFrame(columns=['filename', 'provider', 'device', 'instance', 'iteration', 'sp_peak', 'dp_peak', 'im_peak'])
cpu_ml = pd.DataFrame(columns=['filename', 'provider', 'vcpu(s)', 'device', 'instance', 'iteration', 'accuracy', 'runtime(ms)'])
gpu_ml = pd.DataFrame(columns=['filename', 'provider', 'vcpu(s)', 'device', 'instance', 'iteration', 'accuracy', 'runtime(ms)'])
for f in os.listdir(OUTPUT_DIR):
    try:
        if "ml" in f:
            process_ml_benchmark(f)
        elif "cpu" in f:
            process_cpu_benchmark(f)
        elif "gpu" in f:
            process_gpu_benchmark(f)
        else:
            print("Unrecognized Filename: %s"%f)
    except:
        print(f)
        raise

Unrecognized Filename: .DS_Store
Unrecognized Filename: ocl


In [11]:
def summarize_cpu(df, target):
    df['vcpu(s)'] = df['vcpu(s)'].astype('int')
    df[target] = df[target].astype('float')
    df = df.merge(cpu_price, how='outer')
    df['normalized_runtime'] = df[target] / float(df.groupby(['provider','vcpu(s)'])[target].mean()[0])
    df['cost_per_task'] = df['normalized_runtime'] * df['price']
    df['normalized_cost'] = df['cost_per_task'] / float(df.groupby(['provider','vcpu(s)'])['cost_per_task'].mean()[0])
    return df

In [12]:
def summarize_gpu(df, target='runtime(ms)'):
    df = df.merge(gpu_price, how='outer')
    if isinstance(target, list):
        for t in target:
            df[t] = df[t].astype('float')
            df['normalized_%s'%t] = df[t] / df.groupby(['provider'])[t].mean()[0]
            df['cost_per_perf_%s'%t] = df['normalized_%s'%t] * df['price']
            df['normalized_cost_%s'%t] = df['cost_per_perf_%s'%t] / float(df.groupby(['provider'])['cost_per_perf_%s'%t].mean()[0])
            
    else:
        df[target] = df[target].astype('float')
        df['normalized_runtime'] = df[target] / df.groupby(['provider'])[target].mean()[0]
        df['cost_per_task'] = df['normalized_runtime'] * df['price']
        df['normalized_cost'] = df['cost_per_task'] / float(df.groupby(['provider'])['cost_per_task'].mean()[0])
    return df

In [13]:
summarized_gpu_ml = summarize_gpu(gpu_ml)

In [14]:
summarized_cpu = summarize_cpu(cpu, 'time_per_task(ms)')

In [15]:
summarized_cpu_ml = summarize_cpu(cpu_ml, 'runtime(ms)')

In [16]:
summarized_gpu = summarize_gpu(gpu, ['sp_peak','dp_peak','im_peak'])

In [17]:
summarized_gpu.head()

,filename,provider,device,instance,iteration,sp_peak,dp_peak,im_peak,price,normalized_sp_peak,cost_per_perf_sp_peak,normalized_cost_sp_peak,normalized_dp_peak,cost_per_perf_dp_peak,normalized_cost_dp_peak,normalized_im_peak,cost_per_perf_im_peak,normalized_cost_im_peak
0,amazon_gpu_alt-2.out,amazon,gpu,2,1,3451.88,1375.43,692.15,0.9,1.007785,0.907006,1.007785,0.998132,0.898319,0.998132,0.997601,0.897841,0.997601
1,amazon_gpu_alt-2.out,amazon,gpu,2,2,3449.62,1375.86,690.75,0.9,1.007125,0.906412,1.007125,0.998444,0.898599,0.998444,0.995583,0.896025,0.995583
2,amazon_gpu_alt-2.out,amazon,gpu,2,3,3455.28,1375.69,689.31,0.9,1.008777,0.907900,1.008777,0.998321,0.898488,0.998321,0.993507,0.894157,0.993507
3,amazon_gpu_alt-2.out,amazon,gpu,2,4,3451.55,1375.27,691.22,0.9,1.007688,0.906920,1.007688,0.998016,0.898214,0.998016,0.996260,0.896634,0.996260
4,amazon_gpu_alt-2.out,amazon,gpu,2,5,3469.80,1360.48,691.87,0.9,1.013017,0.911715,1.013017,0.987283,0.888555,0.987283,0.997197,0.897477,0.997197


In [18]:
summarized_gpu_ml.head()

,filename,provider,vcpu(s),device,instance,iteration,accuracy,runtime(ms),price,normalized_runtime,cost_per_task,normalized_cost
0,amazon_4_gpu_ml-1.out,amazon,4,gpu,1,1,0.9870,114515.0,0.9,1.023552,0.921196,1.023552
1,amazon_4_gpu_ml-1.out,amazon,4,gpu,1,2,0.9866,113274.0,0.9,1.012459,0.911213,1.012459
2,amazon_4_gpu_ml-1.out,amazon,4,gpu,1,3,0.9864,113262.0,0.9,1.012352,0.911117,1.012352
3,amazon_4_gpu_ml-1.out,amazon,4,gpu,1,4,0.9870,112760.0,0.9,1.007865,0.907079,1.007865
4,amazon_4_gpu_ml-1.out,amazon,4,gpu,1,5,0.9863,113149.0,0.9,1.011342,0.910208,1.011342


In [24]:
summarized_cpu[summarized_cpu['provider'] == 'google'].sort_values('vcpu(s)')

,filename,provider,vcpu(s),device,instance,iteration,events,runtime(ms),time_per_task(ms),price,normalized_runtime,cost_per_task,normalized_cost
44,google_4_cpu-3.out,google,4,cpu,3,5,13,12094.5,930.346154,0.2088,5.837705,1.218913,7.170075
32,google_4_cpu-2.out,google,4,cpu,2,3,12,11207.5,933.958333,0.2088,5.860371,1.223645,7.197914
33,google_4_cpu-2.out,google,4,cpu,2,4,15,13122.7,874.846667,0.2088,5.489459,1.146199,6.742347
34,google_4_cpu-2.out,google,4,cpu,2,5,16,12101.1,756.318750,0.2088,4.745724,0.990907,5.828865
35,google_4_cpu-2.out,google,4,cpu,2,6,16,12909.5,806.843750,0.2088,5.062756,1.057104,6.218256
36,google_4_cpu-2.out,google,4,cpu,2,7,15,12788.2,852.546667,0.2088,5.349531,1.116982,6.570483
37,google_4_cpu-2.out,google,4,cpu,2,8,14,12175.9,869.707143,0.2088,5.457209,1.139465,6.702737
38,google_4_cpu-2.out,google,4,cpu,2,9,15,12964.4,864.293333,0.2088,5.423239,1.132372,6.661014
39,google_4_cpu-2.out,google,4,cpu,2,10,16,12452.7,778.293750,0.2088,4.883612,1.019698,5.998224
40,google_4_cpu-3.out,google,4,cpu,3,1,16,12543.5,783.968750,0.2088,4.919221,1.027133,6.041961


In [25]:
summarized_cpu[summarized_cpu['provider'] == 'amazon'].sort_values('vcpu(s)')

,filename,provider,vcpu(s),device,instance,iteration,events,runtime(ms),time_per_task(ms),price,normalized_runtime,cost_per_task,normalized_cost
60,amazon_4_cpu-3.out,amazon,4,cpu,3,1,64,10200.3,159.379688,0.17,1.000070,0.170012,1.000070
82,amazon_4_cpu-1.out,amazon,4,cpu,1,3,64,10193.8,159.278125,0.17,0.999433,0.169904,0.999433
83,amazon_4_cpu-1.out,amazon,4,cpu,1,4,64,10194.0,159.281250,0.17,0.999453,0.169907,0.999453
84,amazon_4_cpu-1.out,amazon,4,cpu,1,5,64,10193.5,159.273438,0.17,0.999404,0.169899,0.999404
85,amazon_4_cpu-1.out,amazon,4,cpu,1,6,64,10197.6,159.337500,0.17,0.999806,0.169967,0.999806
86,amazon_4_cpu-1.out,amazon,4,cpu,1,7,64,10191.9,159.248437,0.17,0.999247,0.169872,0.999247
87,amazon_4_cpu-1.out,amazon,4,cpu,1,8,64,10191.7,159.245313,0.17,0.999227,0.169869,0.999227
88,amazon_4_cpu-1.out,amazon,4,cpu,1,9,64,10195.9,159.310937,0.17,0.999639,0.169939,0.999639
89,amazon_4_cpu-1.out,amazon,4,cpu,1,10,64,10193.3,159.270313,0.17,0.999384,0.169895,0.999384
90,amazon_4_cpu-1.out,amazon,4,cpu,1,11,64,10193.7,159.276562,0.17,0.999423,0.169902,0.999423


In [20]:
summarized_cpu_ml.head()

,filename,provider,vcpu(s),device,instance,iteration,accuracy,runtime(ms),price,normalized_runtime,cost_per_task,normalized_cost
0,amazon_4_cpu_ml-3.out,amazon,4,cpu,3,1,0.9876,240842.0,0.17,0.980479,0.166681,0.980479
1,amazon_4_cpu_ml-3.out,amazon,4,cpu,3,2,0.9868,240499.0,0.17,0.979082,0.166444,0.979082
2,amazon_4_cpu_ml-3.out,amazon,4,cpu,3,3,0.9852,243172.0,0.17,0.989964,0.168294,0.989964
3,amazon_4_cpu_ml-3.out,amazon,4,cpu,3,4,0.9868,245767.0,0.17,1.000528,0.170090,1.000528
4,amazon_4_cpu_ml-3.out,amazon,4,cpu,3,5,0.9871,242634.0,0.17,0.987774,0.167922,0.987774


In [46]:
dfs = []
for col in ['sp_peak', 'dp_peak', 'im_peak']:
    df = summarized_gpu.iloc[:, 0:5]
    df['normalized_peak'] = summarized_gpu['normalized_%s'%col]
    df['normalized_cost'] = summarized_gpu['normalized_cost_%s'%col]
    df['task'] = [col.split('_')[0]] * df.shape[0]
    dfs.append(df)
new_gpu = pd.DataFrame(np.concatenate(dfs), columns = dfs[0].columns)

In [47]:
new_gpu.head()

,filename,provider,device,instance,iteration,normalized_peak,normalized_cost,task
0,amazon_gpu_alt-2.out,amazon,gpu,2,1,1.00778,1.00778,sp
1,amazon_gpu_alt-2.out,amazon,gpu,2,2,1.00712,1.00712,sp
2,amazon_gpu_alt-2.out,amazon,gpu,2,3,1.00878,1.00878,sp
3,amazon_gpu_alt-2.out,amazon,gpu,2,4,1.00769,1.00769,sp
4,amazon_gpu_alt-2.out,amazon,gpu,2,5,1.01302,1.01302,sp


In [48]:
for i, df in enumerate([summarized_cpu, new_gpu, summarized_cpu_ml, summarized_gpu_ml]):
    df.to_csv('exp-%d.csv'%i, index=False)